<a href="https://colab.research.google.com/github/WB-Jang/AI-RPA/blob/main/Committee-agent/Word_%EB%8F%99%EC%8B%9C%ED%8E%B8%EC%A7%91_Pdf_%ED%86%B5%ED%95%A9_%ED%94%8C%EB%9E%AB%ED%8F%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Committee 업무 중 The most time-consuming task인 문서 취합 효율화 플랫폼
- 기능 1 : 여러 Word 파일들을 한 화면에서 동시 편집 가능
- 기능 2 : 일부 수정된 Word 파일을 Local PC에 저장 혹은 작성 부서에 e-mail 전송
- 기능 3 : 최종 수정된 Word 파일들을 pdf로 통합
- 기능 4 : LLM을 활용하여, 오타 및 논리적 오류가 보이는 부분 Early-alert

In [1]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.9 MB/s eta 0:00:00


In [6]:
%%writefile app.py
# Streamlit 어플 구성
import streamlit as st
import os

def main():
  st.set_page_config(page_title="Word 취합 및 교정, Pdf 통합 플랫폼", layout='wide')

  if "analysis_cache" not in st.session_state:
    st.session_state.analysis_cache = {}
  if "summary_cache" not in st.session_state:
    st.session_state.summary_cache = {}

  st.sidebar.header("설정")
  folder = st.sidebar.text_input("Word 파일 폴더 경로", value="")
  output_pdf = st.sidebar.text_input("최종 통합 pdf 저장 경로", value=os.path.join(os.getcwd(), 'Integrated.pdf'))
  st.title("Word 문서 취합/교정/통합 PDF 생성 tool")

  if not folder or not os.path.isdir(folder):
    st.warning("좌측에 Word 파일들이 모여 있는 폴더 경로를 입력하세요.") # 단, streamlit은 wsl에서 돌아가고 있으므로, /mnt/c/Users/...과 같은 입력이 필요
    return

  files = list_word_files(folder)
  if not files:
    st.warning(f"{folder} 폴더에서 .doc / .docx 파일을 찾지 못했습니다.")
    return

  st.sidebar.markdown("---")
  st.sidebar.write(f"파일 개수 : **{len(files)}**")

  if st.sidebar.button("모든 문서를 통합하여 PDF 생성", type="primary"):
    try:
      with st.spinner("WORD 파일들을 PDF로 병합 중..."):
        merge_docs_to_pdf(files, output_pdf)
      st.sidebar.success(f"PDF 생성 완료 : {output_pdf}")
    except Exception as e:
      st.sidebar.error(f"PDF 생성 중 오류 : {e}")

  st.markdown("### 문서 목록")

  for path in sorted(files):
    filename = os.path.basename(path)
    with st.expander(filename, expanded=False):
      col1, col2, col3 = st.columns(3)
      with col1:
        if st.button("Word로 열기", key=f"open_{path}"):
          try:
            open_in_word(path)
            st.success("Word에서 문서를 열었습니다")
          except Exception as e:
            st.error(f"Word에서 문서를 열지 못했습니다 : {e}")
      with col2:
        if st.button("맞춤법/논리 오류 검사", key=f"analyze_{path}"):
          try:
            text = extract_text_from_docx(path)
            with st.spinner("LLM으로 문서 검사 중..."):
              result = llm_analyze_text(text)
            st.session_state.analysis_cache[path] = result
          except Exception as e:
            st.error(f"LLM으로 문서 검사 중 오류 : {e}")
      with col3:
        if st.button("요약 생성", key=f"summarize_{path}"):
          try:
            text = extract_text_from_docx(path)
            with st.spinner("LLM으로 요약 중..."):
              result = llm_summarize_text(text)
            st.session_state.summary_cache[path] = result
          except Exception as e:
            st.error(f"요약 중 오류 : {e}")
      st.markdown("---")
      st.write("**LLM 검사 결과**")
      if path in st.session_state.analysis_cache:
        st.text_area(
            "LLM 검사 결과 검토 필요 사항"
            ,st.session_state.analysis_cache[path]
            ,key=f"analysis_area_{path}"
            ,height=200
            ,)
      else:
        st.info("LLM 검사 결과가 없습니다")
      st.write("**LLM 요약 결과**")
      if path in st.session_state.summary_cache:
        st.text_area(
            "LLM 요약 결과"
            ,st.session_state.summary_cache[path]
            ,key=f"summary_area_{path}"
            ,height=200
            ,)
      else:
        st.info("LLM 요약 결과가 없습니다")

  st.markdown("---")
  st.caption(
      "- 각 부서에서 보내온 Word 파일은 이 화면에서 확인하고, 'Word로 열기' 기능으로 직접 수정합니다. \n"
      "- 수정이 모두 끝난 뒤 좌측의 '모든 문서를 통합하여 PDF 생성' 기능으로 최종 통합 PDF를 생성합니다"
  )


if __name__ == "__main__":
  main()

Overwriting app.py


In [3]:
!pip install pyngrok

In [7]:
import threading
import subprocess
import time
from pyngrok import ngrok

def run_streamlit():
  cmd = ["streamlit", "run", "app.py", "--server.port", "8501", "--server.address=0.0.0.0"]
  subprocess.run(cmd)

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

time.sleep(5)

NGROK_AUTH_TOKEN = "31LySo6cMJ2KtrV8l3zVzZz0TGx_2TnffbAQuaW5vVffdPxf4"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

tunnel = ngrok.connect(8501, "http")
print("외부 접속 URL : ", tunnel.public_url)

외부 접속 URL :  https://e11bf8a1f521.ngrok-free.app
